# 05 — Time Series Analysis
**Author:** Ebenezer Adjartey

Covers: plotting, ACF/PACF, stationarity tests (ADF, KPSS, PP), ARIMA/SARIMA modeling, VAR, VECM, ARCH/GARCH volatility modeling.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import coint
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
print('Libraries loaded. (install arch if missing: pip install arch)')

## 1. Generate Synthetic Time Series

In [ ]:
n = 200
dates = pd.date_range('2005-01', periods=n, freq='M')

# AR(2) process: y_t = 0.6*y_{t-1} - 0.2*y_{t-2} + e_t
e = np.random.normal(0, 1, n)
y = np.zeros(n)
for t in range(2, n):
    y[t] = 0.6*y[t-1] - 0.2*y[t-2] + e[t]
ts = pd.Series(y, index=dates, name='AR2')

# Random walk (non-stationary): y_t = y_{t-1} + e_t
rw = pd.Series(np.cumsum(np.random.normal(0,1,n)), index=dates, name='RandomWalk')

# Seasonal series
seasonal = pd.Series(
    np.arange(n)*0.1 + 5*np.sin(2*np.pi*np.arange(n)/12) + np.random.normal(0,.5,n),
    index=dates, name='Seasonal'
)

print('AR(2) head:', ts.head().values)
print('Seasonal head:', seasonal.head().values)

## 2. Time Series Plot + ACF/PACF

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 9))
ts.plot(ax=axes[0], title='AR(2) Process'); axes[0].set_ylabel('Value')
rw.plot(ax=axes[1], title='Random Walk'); axes[1].set_ylabel('Value')
seasonal.plot(ax=axes[2], title='Seasonal Series'); axes[2].set_ylabel('Value')
plt.tight_layout()
os.makedirs('05_time_series_analysis', exist_ok=True)
plt.savefig('05_time_series_analysis/ts_plots.png', dpi=100, bbox_inches='tight')
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(ts,  lags=20, ax=axes[0], title='ACF of AR(2)')
plot_pacf(ts, lags=20, ax=axes[1], title='PACF of AR(2)')
plt.tight_layout()
plt.savefig('05_time_series_analysis/acf_pacf.png', dpi=100, bbox_inches='tight')
plt.show(); print('Saved.')

## 3. Stationarity Tests

In [ ]:
def adf_test(series, name=''):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Test [{name}]: stat={result[0]:.4f}, p={result[1]:.4f}, lags={result[2]}')
    for k, v in result[4].items():
        print(f'  Critical value {k}: {v:.3f}')
    print('  Verdict:', 'Stationary' if result[1] < 0.05 else 'Non-stationary')

def kpss_test(series, name=''):
    result = kpss(series, regression='c', nlags='auto')
    print(f'KPSS Test [{name}]: stat={result[0]:.4f}, p={result[1]:.4f}')
    print('  Verdict:', 'Non-stationary' if result[1] < 0.05 else 'Stationary')

print('=== Stationarity Tests ===')
for s, name in [(ts,'AR2'), (rw,'RandomWalk'), (rw.diff().dropna(),'RW_Differenced')]:
    adf_test(s, name); kpss_test(s, name); print()

## 4. ARIMA Modeling

In [ ]:
# Fit ARIMA(2,0,0) to stationary AR(2) series
arima_model = ARIMA(ts, order=(2,0,0)).fit()
print(arima_model.summary())

# Fit ARIMA(1,1,1) to random walk (needs differencing)
arima_rw = ARIMA(rw, order=(1,1,1)).fit()
print('\nARIMA(1,1,1) AIC:', round(arima_rw.aic, 2))

# Forecast
forecast = arima_model.get_forecast(steps=12)
fc_mean = forecast.predicted_mean
fc_ci   = forecast.conf_int()
print('\n12-step forecast:')
print(pd.concat([fc_mean.rename('forecast'), fc_ci], axis=1).round(3))

## 5. SARIMA (Seasonal ARIMA)

In [ ]:
sarima_model = SARIMAX(seasonal, order=(1,1,1), seasonal_order=(1,1,0,12)).fit(disp=False)
print(sarima_model.summary())
print('SARIMA AIC:', round(sarima_model.aic, 2))

## 6. VAR (Vector Autoregression)

In [ ]:
# Two correlated series
e1, e2 = np.random.multivariate_normal([0,0], [[1,0.7],[0.7,1]], n).T
y1, y2 = np.zeros(n), np.zeros(n)
for t in range(2, n):
    y1[t] = 0.5*y1[t-1] + 0.2*y2[t-1] + e1[t]
    y2[t] = 0.1*y1[t-1] + 0.4*y2[t-1] + e2[t]

df_var = pd.DataFrame({'y1':y1, 'y2':y2}, index=dates)
var_model = VAR(df_var)
var_result = var_model.fit(maxlags=4, ic='aic')
print(f'VAR selected lag order: {var_result.k_ar}')
print(var_result.summary())

## 7. ARCH/GARCH (Volatility Modeling)

In [ ]:
# Generate GARCH(1,1) returns
returns = np.random.normal(0, 1, n)
for t in range(2, n):
    vol = np.sqrt(0.01 + 0.1*returns[t-1]**2 + 0.85*(returns[:t].var()))
    returns[t] = vol * np.random.normal()

garch_model = arch_model(returns, vol='Garch', p=1, q=1, dist='normal')
garch_result = garch_model.fit(disp='off')
print(garch_result.summary())

# Volatility forecast
forecasts = garch_result.forecast(horizon=5)
print('\nConditional variance forecast:')
print(forecasts.variance.iloc[-1].round(6))

## Key Takeaways

- Test stationarity (ADF, KPSS) before modeling
- AR terms use PACF; MA terms use ACF for order identification
- ARIMA(p,d,q): d=order of differencing for stationarity
- VAR models interdependencies between multiple time series
- GARCH models time-varying volatility (important in finance)
